In [2]:
# 
# Aroldis Chapman and the 4 Seam Fastball
# He owns most of the top 15 fastest pitches since 2015
# All 4 seams
# He is a closer
# What are the results of these pitches?
# Where do they go?
# What are his stats
# How are his other pitches?
# are neg X's inside?
    # yeah

In [3]:
# DONE libraries
# TODO show top 15 query and explain
# DONE show positions of these pitches
# TODO look into some of just his stats
# TODO pull his non 4F pitches and compare
# DONE find if neg X's are inside


In [4]:
import pandas_gbq as pdgbq
import pandas as pd
import plotly.express as px

In [5]:
top15query ='''
-- Temp function to get year

CREATE TEMP FUNCTION getYear(x DATE)
    RETURNS INT64
    AS (EXTRACT(year FROM x));

SELECT
    getYear(gd.game_date) AS year,
    pv.pitcher_full,
    pr.pitch_name,
    pv.release_speed,
    pv.release_spin_rate,
    pv.plate_x,
    pv.plate_z
FROM 
    `valuesheet.MLB.pitching_view` AS pv
INNER JOIN 
    `valuesheet.MLB.pitch_ref` AS pr ON pv.pitch_type = pr.pitch_type
INNER JOIN 
    `valuesheet.MLB.game_data` AS gd ON pv.game_pk = gd.game_pk
ORDER BY
    release_speed DESC
LIMIT 15 
'''

In [6]:
fastestpitches = pdgbq.read_gbq(query=top15query,project_id='valuesheet')

Downloading: 100%|██████████| 15/15 [00:00<00:00, 88.22rows/s]


In [7]:
fastestpitches.head(15)

,year,pitcher_full,pitch_name,release_speed,release_spin_rate,plate_x,plate_z
0,2016,Aroldis Chapman,4-Seam Fastball,105.7,2617.0,-2.10,1.57
1,2016,Aroldis Chapman,4-Seam Fastball,105.4,NaN,1.08,3.00
2,2016,Aroldis Chapman,4-Seam Fastball,105.2,NaN,-0.90,1.87
3,2016,Aroldis Chapman,4-Seam Fastball,105.1,NaN,0.26,1.99
4,2016,Aroldis Chapman,4-Seam Fastball,105.1,2698.0,-1.56,2.55
5,2016,Aroldis Chapman,4-Seam Fastball,105.1,NaN,-0.47,0.69
6,2016,Aroldis Chapman,4-Seam Fastball,105.0,2500.0,-1.30,1.38
7,2018,Jordan Hicks,Sinker,105.0,2289.0,-2.43,3.30
8,2018,Jordan Hicks,Sinker,105.0,2197.0,-1.28,2.29
9,2016,Aroldis Chapman,4-Seam Fastball,104.8,2654.0,-1.53,3.35


In [8]:
fig = px.scatter(fastestpitches, x="plate_x", y="plate_z", color="pitch_name",
                 title="Fastest Pitches, 2015 - Present",
                 labels={"plate_x":"Inside/Outside","plate_z":"High/Low"},
                 height=500,
                 width=500)
                     
fig.update_layout(legend_title_text='Pitch')

In [9]:
chapmanPitches = '''
-- Temp function to get year

CREATE TEMP FUNCTION getYear(x DATE)
    RETURNS INT64
    AS (EXTRACT(year FROM x));

--Get chapman's fastest pitches grouped by pitch and year

WITH pitches AS (SELECT
    getYear(gd.game_date) AS year,
    max(pv.release_speed) AS fastest_pitch,
    pv.pitch_type
FROM
    `valuesheet.MLB.pitching_view` AS pv
LEFT JOIN 
    `valuesheet.MLB.game_data` AS gd ON pv.game_pk = gd.game_pk
WHERE 
    pv.pitcher_full = "Aroldis Chapman"
GROUP BY
    year,
    pv.pitch_type
ORDER BY
    fastest_pitch DESC),

-- Rank his fastest pitches by pitch

pitchrank AS (SELECT 
    year,
    pitch_type,
    fastest_pitch,
    RANK() OVER(PARTITION BY pitch_type ORDER BY fastest_pitch DESC) AS Rank
FROM
    pitches
ORDER BY 
    fastest_pitch DESC,
    Rank),

-- Rank his fastest pitches by year

yearrank AS (SELECT
    year,
    fastest_pitch,
    pitch_type,
    RANK() OVER(PARTITION BY year ORDER BY fastest_pitch DESC) AS rankyear
FROM 
    pitchrank
ORDER BY
    rankyear)

-- Select the best 1 rank pitches by year

SELECT
    yr.year,
    yr.fastest_pitch,
    pr.pitch_name
FROM 
    yearrank AS yr
LEFT JOIN 
    `valuesheet.MLB.pitch_ref` AS pr ON yr.pitch_type = pr.pitch_type
WHERE 
    rankyear = 1
ORDER BY 
    fastest_pitch DESC
'''

In [10]:
chapmanBest = pdgbq.read_gbq(query=chapmanPitches,project_id='valuesheet')

chapmanBest.head(10)

Downloading: 100%|██████████| 7/7 [00:00<00:00, 42.08rows/s]


,year,fastest_pitch,pitch_name
0,2016,105.7,4-Seam Fastball
1,2018,104.4,Sinker
2,2017,104.2,4-Seam Fastball
3,2015,104.1,4-Seam Fastball
4,2021,103.4,Sinker
5,2019,102.7,Sinker
6,2020,101.5,4-Seam Fastball


In [11]:
chapmanPitchesNoFF = '''
-- Declare vars
--  Don't want the 4 seam at a certain point in this query

DECLARE seam4 STRING DEFAULT 'FF';

-- Temp function to get year

CREATE TEMP FUNCTION getYear(x DATE)
    RETURNS INT64
    AS (EXTRACT(year FROM x));

--Get chapman's fastest pitches grouped by pitch and year

WITH pitches AS (SELECT
    getYear(gd.game_date) AS year,
    max(pv.release_speed) AS fastest_pitch,
    pv.pitch_type
FROM
    `valuesheet.MLB.pitching_view` AS pv
LEFT JOIN 
    `valuesheet.MLB.game_data` AS gd ON pv.game_pk = gd.game_pk
WHERE 
    pv.pitcher_full = "Aroldis Chapman"
    AND pv.pitch_type NOT IN (seam4)
GROUP BY
    year,
    pv.pitch_type
ORDER BY
    fastest_pitch DESC
),

-- Rank his fastest pitches by pitch

pitchrank AS (SELECT 
    year,
    pitch_type,
    fastest_pitch,
    RANK() OVER(PARTITION BY pitch_type ORDER BY fastest_pitch DESC) AS Rank
FROM
    pitches
ORDER BY 
    fastest_pitch DESC,
    Rank),

-- Rank his fastest pitches by year

yearrank AS (SELECT
    year,
    fastest_pitch,
    pitch_type,
    RANK() OVER(PARTITION BY year ORDER BY fastest_pitch DESC) AS rankyear
FROM 
    pitchrank
ORDER BY
    rankyear)

-- Select the best 1 rank pitches by year

SELECT
    yr.year,
    yr.fastest_pitch,
    pr.pitch_name
FROM 
    yearrank AS yr
LEFT JOIN 
    `valuesheet.MLB.pitch_ref` AS pr ON yr.pitch_type = pr.pitch_type
WHERE 
    rankyear = 1
ORDER BY 
    fastest_pitch DESC
'''

In [12]:
chapmanBestNoFF = pdgbq.read_gbq(query=chapmanPitchesNoFF,project_id='valuesheet')

chapmanBestNoFF.head(10)

Downloading: 100%|██████████| 7/7 [00:00<00:00, 35.37rows/s]


,year,fastest_pitch,pitch_name
0,2018,104.4,Sinker
1,2021,103.4,Sinker
2,2017,102.8,Sinker
3,2019,102.7,Sinker
4,2020,101.1,Sinker
5,2016,94.0,Slider
6,2015,92.1,Changeup


In [14]:
chapmanBestNoFF.to_html(buf='main.html')